# 实验一
自定义深度卷积申请网络，并在Kaggle猫/狗数据集上进行训练和测试

## 1.加载keras模块

In [1]:
#加载ImageDataGenerator模块
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
#加载定义CNN的layers、activate function等模块
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
#以下为将预测score转换为类别信息的to_categorical、图像预处理等函数
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
import numpy as np

Using TensorFlow backend.


### 定义CNN网络结构
首先尝试：自定义深度CNN结构

然后尝试：定义Alex/VGG网络结构


In [2]:
#首先定义图像的长和宽，目的是告诉网络输入图像矩阵的大小，便于搭建网络结构
img_width = 150
img_height = 150

#以下为确认图像的通道分量
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
#在此定义网络结构    
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation = 'relu',input_shape=input_shape))
# input_shape = ???
# kernel_size = ???    根据实验 5来看 320 ==32*9 +32*1  ///9 =3*3 kernel——size   1 = bias 
#model.add(MaxPool2D(pool_size=(2,2))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation = 'relu'))
#model.add(MaxPool2D(pool_size=(2,2))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64,kernel_size=(3,3),activation = 'relu'))
#model.add(MaxPool2D(pool_size=(2,2))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))

model.add(Dropout(0.5))
model.add(Dense(1,activation = 'sigmoid'))
#在此调用compile函数定义loss function，optimizer和metrics


#optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model.compile(optimizer = optimizer,loss = 'binary_crossentropy', metrics=["accuracy"])
model.compile(optimizer ='rmsprop',loss = 'binary_crossentropy', metrics=["accuracy"])
'''
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
'''

"\nlearning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', \n                                            patience=3, \n                                            verbose=1, \n                                            factor=0.5, \n                                            min_lr=0.00001)\n"

### 查看model架构



In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

### 定义ImageDataGenerator


In [51]:
from tensorflow.keras.preprocessing import image
#import pandas as pd
#指定train和validation数据路径
#train_path = 'D:\AI\11.6\dogs-vs-cats\train'
#validation_path= 'D:\AI\11.6\dogs-vs-cats\validation'
train_data_dir = r'D:\AI\11.6\dogs-vs-cats\train'
validation_data_dir = r'D:\AI\11.6\dogs-vs-cats\validation'
#指定train和validation数据规模
nb_train_samples = 10835
nb_validation_samples = 4000




In [52]:
epochs = 1
batch_size = 20

#声明ImageDataGenerator类型的train_datagen用于对training图像进行数据像素归一化和扩增
train_datagen=ImageDataGenerator(rescale=1./255, shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

#声明ImageDataGenerator类型的test_datagen用于对validation图像进行像素归一化
test_datagen = ImageDataGenerator(rescale=1./255)
#调用flow_from_directory生成train_generator
train_generator = train_datagen.flow_from_directory(train_data_dir,target_size=(img_width, img_height),
                                                    batch_size=batch_size,class_mode='binary')
#调用flow_from_directory生成validation_generator
validation_generator = test_datagen.flow_from_directory(validation_data_dir,target_size=(img_width, img_height),
                                                        batch_size=batch_size,class_mode='binary')

Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [53]:
#调用并设置fit_generator函数训练模型
#history = model.fit(train_datagen,test_datagen, batch_size = batch_size,
                             #epochs = epochs , validation_data = (X_val,Y_val))
history = model.fit_generator(train_generator,steps_per_epoch = nb_train_samples // batch_size,
                             epochs = epochs , validation_data = validation_generator,validation_steps=nb_validation_samples // batch_size)

541/541 [==============================] - 158s 292ms/step - loss: 0.6613 - accuracy: 0.6178 - val_loss: 0.5854 - val_accuracy: 0.6995


### 使用训练后模型预测图像





In [46]:
import cv2


In [ ]:
#调用opencv的imread函数和resize函数读取测试样本并进行尺寸缩放，此处路径可更换
img = cv2.resize(cv2.imread(r'D:/AI/11.1/7.jpg'), (img_width, img_height)).astype(np.float32)

#对图像像素矩阵进行tensor类型转换
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
#用训练好的模型对测试样本进行预测
score = model.predict(x)
print(score)

In [45]:
#调用并设置fit_generator函数训练模型
#history = model.fit(train_datagen,test_datagen, batch_size = batch_size,
                             #epochs = epochs , validation_data = (X_val,Y_val))
history = model.fit_generator(train_generator,steps_per_epoch = nb_train_samples // batch_size,
                             epochs = epochs , validation_data = validation_generator,validation_steps=nb_validation_samples // batch_size)

541/541 [==============================] - 209s 386ms/step - loss: 7.1607 - accuracy: 0.5330 - val_loss: 7.6666 - val_accuracy: 0.5000
